<a href="https://colab.research.google.com/github/jx-dohwan/BERT-FP_MRS/blob/main/Post_Pretrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Post-training

## 라이브러리 설치

In [1]:
!pip install torch
!pip install transformers==4.25.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 64.3 MB/s eta 0:00:00


In [2]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



## 데이터 다운로드

### 스마일 스타일 데이터세트(for 대화 및 스타일 트랜스퍼)

In [3]:
!git clone https://github.com/smilegate-ai/korean_smile_style_dataset
# https://corpus.korean.go.kr/
# https://aihub.or.kr/aihub-data/natural-language/about

Cloning into 'korean_smile_style_dataset'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 81 (delta 21), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (81/81), 2.57 MiB | 6.63 MiB/s, done.


In [4]:
import pandas as pd
df = pd.read_csv('/content/korean_smile_style_dataset/smilestyle_dataset.tsv', sep='\t')
df.to_csv('./korean_smile_style_dataset/smile.csv', index=False)

In [5]:
!head -3 './korean_smile_style_dataset/smile.csv'

formal,informal,android,azae,chat,choding,emoticon,enfp,gentle,halbae,halmae,joongding,king,naruto,seonbi,sosim,translator
안녕하세요. 저는 고양이 6마리 키워요.,안녕! 나는 고양이 6마리 키워.,휴먼. 반갑다. 안드로이드는. 고양이. 6마리. 소유중.,아이고 안녕하십니까~ 나는 그냥 고양이 6마리 키우고 있는 사람이여,하잉ㅋㅋ 나 떼걸룩 6마리 키운다!,ㅎㅇ 나 주인님 6마리 모심 ㅋㅋ,안녕!! >< 나는 고양이😺를 ➏ 마리 키우고있어!! 0_0,안녕안녕~! 나 고양이 6마리나 키운다? 완전 대박이징~,"안녕하십니까,, 저는 고양이 6마리 키웁니다.",안녕하신가~... 난 지금 고양이를 6마리 키우고 있다네,하유 시벌것 괭이놈 6마리 키우는데 힘들어 죽겟네,안녕하냐 ㅡㅡ 나 씹냥이 6마리나 키운다 하;,반갑소. 짐은 고양이를 6마리나 키우오.,안녕하냐니깐! 난 고양이를 6마리 키우고있다니깐!,안녕하시오! 소인은 고양이를 6마리 키우고 있소!,안녕… 난 고양이 6마리 키워 ㅠㅠ,반가운. 나는 6마리의 고양이를 소지하고 있다.
고양이를 6마리나요? 키우는거 안 힘드세요?,고양이를 6마리나? 키우는거 안 힘들어?,고양이. 6마리. 양육. 번거로운가.,아니 무슨 고양이를 6마리나? 거 키우는 거 안 힘든가?,엥? 6마리나? 안힘듬?ㅋㅋㅋㅋ,6마리? 에바아니냐 안 힘듦?,고양이를 6마리나?!! w(ﾟДﾟ)w 키우는거 안 힘듬?? (⊙_⊙;),고양이를 6마리나? 완전 대박~ 키우는 거 안 힘들어?!,"고양이를 6마리나 키우십니까? 안 힘드신지,,",고양이를 6마리나? 키우는거 힘들지 않는가?,니기럴 털만 날리는 거 키우기 안 힘들데?,아니 고양이를 6마리나? 안힘드냐?,고양이를 6마리나? 키우는게 수고스럽진 않소?,고양이를 6마리나? 키우는거 힘들지 않냐니깐?,고양이를 6마리나 키우고 있는 것이오? 힘들지 않소?,고양이..6마리나? ㅠ 키우는건 혹시 안힘들어..?,6마리의

## 데이터 가공하기

In [6]:
import csv

data_path = './korean_smile_style_dataset/smile.csv'
f = open(data_path, 'r')
rdr = csv.reader(f)

for line in rdr:
    print(line)
    break

['formal', 'informal', 'android', 'azae', 'chat', 'choding', 'emoticon', 'enfp', 'gentle', 'halbae', 'halmae', 'joongding', 'king', 'naruto', 'seonbi', 'sosim', 'translator']


In [7]:
# 데이터 저장
def split(session):
    final_data = []
    split_session = []
    for line in session:
        split_session.append(line)
        final_data.append(split_session[:])
    return final_data

f = open(data_path, 'r')
rdr = csv.reader(f)

""" 세션 데이터 저장할 것"""
session_dataset = []
session = []

""" 실제 데이터 저장 방식 """
for i, line in enumerate(rdr):
    if i == 0:
        header = line
    else:
        utt = line[0] #formal
        if utt.strip() != '':
            session.append(utt)
        else:
            """ 세션 데이터 저장 """
            session_dataset.append(session)
            session = []
""" 마지막 세션 저장 """
session_dataset.append(session)
f.close()

In [8]:
session_dataset[0]

['안녕하세요. 저는 고양이 6마리 키워요.',
 '고양이를 6마리나요? 키우는거 안 힘드세요?',
 '제가 워낙 고양이를 좋아해서 크게 힘들진 않아요.',
 '가장 나이가 많은 고양이가 어떻게 돼요?',
 '여섯 살입니다. 갈색 고양이에요.',
 '그럼 가장 어린 고양이가 어떻게 돼요?',
 '한 살입니다. 작년에 분양 받았어요.',
 '그럼 고양이들끼리 안 싸우나요?',
 '저희 일곱은 다같이 한 가족입니다. 싸우는 일은 없어요.']

In [9]:
len(session_dataset)

236

## 모델을 학습하기 위한 loss 

## MLM을 위한 입력

In [10]:
 from transformers import AutoTokenizer
 tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base')

In [11]:
tokenizer.special_tokens_map_extended

{'bos_token': '[CLS]',
 'eos_token': '[SEP]',
 'unk_token': '[UNK]',
 'sep_token': '[SEP]',
 'pad_token': '[PAD]',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]'}